# Import libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics 
import src.Clean_fnct as clean
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup

# Open dataset

In [4]:
stay = pd.read_csv('./data/listings.csv', encoding = 'ISO - 8859-1')
stay.head(10)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,6369,"Rooftop terrace room with ensuite bathroom, Airc.",13660,Simon,ChamartÃ­n,HispanoamÃ©rica,40.45628,-3.67763,Private room,70,1,65,2019-09-04,0.56,1,53
1,21853,Bright and airy room,83531,Abdel,Latina,CÃ¡rmenes,40.40341,-3.74084,Private room,17,4,33,2018-07-15,0.55,2,48
2,24805,Gran Via Studio Madrid,101471,Iraido,Centro,Universidad,40.42202,-3.70395,Entire home/apt,80,5,2,2017-07-03,0.03,1,354
3,24836,"Select the Madrid more ""cool"".",101653,Tenty,Centro,Justicia,40.41995,-3.69764,Entire home/apt,115,3,64,2019-07-07,0.63,1,301
4,26825,Single Room whith private Bathroom,114340,Agustina,Arganzuela,Legazpi,40.38985,-3.69011,Private room,25,2,136,2019-07-10,1.19,1,337
5,30320,Great Vacational Apartments,130907,Dana,Centro,Sol,40.41552,-3.70346,Entire home/apt,65,5,168,2019-07-09,1.50,3,322
6,30959,Beautiful loft in Madrid Center,132883,Angela,Centro,Embajadores,40.41111,-3.70269,Entire home/apt,54,3,8,2017-05-30,0.15,1,0
7,40916,Holiday Apartment Madrid Center,130907,Dana,Centro,Universidad,40.42096,-3.70428,Entire home/apt,90,3,38,2019-07-07,0.35,3,337
8,59923,HabitaciÃ³n para fÃ©minas en el centro de Madrid,288380,Patricia,Centro,Palacio,40.41971,-3.71256,Private room,40,1,14,2018-12-09,0.14,1,363
9,62423,MAGIC ARTISTIC HOUSE IN THE CENTER OF MADRID,303845,Arturo,Centro,Justicia,40.42080,-3.69878,Private room,45,1,57,2019-07-29,3.43,2,352


In [5]:
stay.shape

(20837, 16)

# Limpieza dataset

 - What columns in the dataset?

In [6]:
stay.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

- Columns that aren't relevant

In [7]:
stay = stay.drop(['minimum_nights','name', 'last_review', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365'], axis = 1 )

In [8]:
stay.head(10)

,id,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,number_of_reviews
0,6369,13660,Simon,ChamartÃ­n,HispanoamÃ©rica,40.45628,-3.67763,Private room,70,65
1,21853,83531,Abdel,Latina,CÃ¡rmenes,40.40341,-3.74084,Private room,17,33
2,24805,101471,Iraido,Centro,Universidad,40.42202,-3.70395,Entire home/apt,80,2
3,24836,101653,Tenty,Centro,Justicia,40.41995,-3.69764,Entire home/apt,115,64
4,26825,114340,Agustina,Arganzuela,Legazpi,40.38985,-3.69011,Private room,25,136
5,30320,130907,Dana,Centro,Sol,40.41552,-3.70346,Entire home/apt,65,168
6,30959,132883,Angela,Centro,Embajadores,40.41111,-3.70269,Entire home/apt,54,8
7,40916,130907,Dana,Centro,Universidad,40.42096,-3.70428,Entire home/apt,90,38
8,59923,288380,Patricia,Centro,Palacio,40.41971,-3.71256,Private room,40,14
9,62423,303845,Arturo,Centro,Justicia,40.42080,-3.69878,Private room,45,57


- We remove the null values

In [9]:
stay.isnull().sum().sort_values(ascending=False)

host_name              15
id                      0
host_id                 0
neighbourhood_group     0
neighbourhood           0
latitude                0
longitude               0
room_type               0
price                   0
number_of_reviews       0
dtype: int64

In [10]:
stay.dropna()

,id,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,number_of_reviews
0,6369,13660,Simon,ChamartÃ­n,HispanoamÃ©rica,40.45628,-3.67763,Private room,70,65
1,21853,83531,Abdel,Latina,CÃ¡rmenes,40.40341,-3.74084,Private room,17,33
2,24805,101471,Iraido,Centro,Universidad,40.42202,-3.70395,Entire home/apt,80,2
3,24836,101653,Tenty,Centro,Justicia,40.41995,-3.69764,Entire home/apt,115,64
4,26825,114340,Agustina,Arganzuela,Legazpi,40.38985,-3.69011,Private room,25,136
...,...,...,...,...,...,...,...,...,...,...
20832,38725567,226958922,CristÃ³bal,ChamartÃ­n,Castilla,40.47847,-3.68395,Entire home/apt,100,0
20833,38726351,178608863,Rostyslav,Puente de Vallecas,Palomeras Sureste,40.38533,-3.63968,Private room,24,0
20834,38727654,182062359,Jose Maciel,Centro,Universidad,40.42182,-3.70336,Entire home/apt,75,0
20835,38728727,31425029,Valeska,Centro,Embajadores,40.40896,-3.70107,Entire home/apt,45,0


- Are there any duplicates?

In [11]:
stay.duplicated().sum()

0

- New size of the dataset: from (20837, 16) to (20837, 10)

In [12]:
stay.shape

(20837, 10)

In [13]:
stay.head()

,id,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,number_of_reviews
0,6369,13660,Simon,ChamartÃ­n,HispanoamÃ©rica,40.45628,-3.67763,Private room,70,65
1,21853,83531,Abdel,Latina,CÃ¡rmenes,40.40341,-3.74084,Private room,17,33
2,24805,101471,Iraido,Centro,Universidad,40.42202,-3.70395,Entire home/apt,80,2
3,24836,101653,Tenty,Centro,Justicia,40.41995,-3.69764,Entire home/apt,115,64
4,26825,114340,Agustina,Arganzuela,Legazpi,40.38985,-3.69011,Private room,25,136


- We rename neighbourhood_group to district

In [26]:
stay.rename(columns={'neighbourhood_group':'district'},inplace=True)

In [27]:
stay.head()

,id,host_id,host_name,room_type,disctricts,neighbourhoods,price,number_of_reviews,latitude,longitude
0,6369,13660,Simon,Private room,Chamartín,Hispanoamérica,70,65,40.45628,-3.67763
1,21853,83531,Abdel,Private room,Latina,Cármenes,17,33,40.40341,-3.74084
2,24805,101471,Iraido,Entire home/apt,Centro,Universidad,80,2,40.42202,-3.70395
3,24836,101653,Tenty,Entire home/apt,Centro,Justicia,115,64,40.41995,-3.69764
4,26825,114340,Agustina,Private room,Arganzuela,Legazpi,25,136,40.38985,-3.69011


- Some names in the district columns are with special characters. We will modify them.

In [16]:
stay['disctrict'].unique()

array(['ChamartÃ\xadn', 'Latina', 'Centro', 'Arganzuela', 'Salamanca',
       'TetuÃ¡n', 'Fuencarral - El Pardo', 'Ciudad Lineal',
       'ChamberÃ\xad', 'Villaverde', 'Hortaleza', 'Moncloa - Aravaca',
       'Carabanchel', 'Retiro', 'San Blas - Canillejas',
       'Villa de Vallecas', 'Barajas', 'Usera', 'Puente de Vallecas',
       'Moratalaz', 'VicÃ¡lvaro'], dtype=object)

In [17]:
stay['disctricts'] = stay['disctrict'].apply(clean.vowels)
stay.head()

,id,host_id,host_name,disctrict,neighbourhood,latitude,longitude,room_type,price,number_of_reviews,disctricts
0,6369,13660,Simon,ChamartÃ­n,HispanoamÃ©rica,40.45628,-3.67763,Private room,70,65,Chamartín
1,21853,83531,Abdel,Latina,CÃ¡rmenes,40.40341,-3.74084,Private room,17,33,Latina
2,24805,101471,Iraido,Centro,Universidad,40.42202,-3.70395,Entire home/apt,80,2,Centro
3,24836,101653,Tenty,Centro,Justicia,40.41995,-3.69764,Entire home/apt,115,64,Centro
4,26825,114340,Agustina,Arganzuela,Legazpi,40.38985,-3.69011,Private room,25,136,Arganzuela


- Same thing happens with the neighbourhood column. We will apply the same function.

In [18]:
stay['neighbourhoods'] = stay['neighbourhood'].apply(clean.vowels)
stay.head()

,id,host_id,host_name,disctrict,neighbourhood,latitude,longitude,room_type,price,number_of_reviews,disctricts,neighbourhoods
0,6369,13660,Simon,ChamartÃ­n,HispanoamÃ©rica,40.45628,-3.67763,Private room,70,65,Chamartín,Hispanoamérica
1,21853,83531,Abdel,Latina,CÃ¡rmenes,40.40341,-3.74084,Private room,17,33,Latina,Cármenes
2,24805,101471,Iraido,Centro,Universidad,40.42202,-3.70395,Entire home/apt,80,2,Centro,Universidad
3,24836,101653,Tenty,Centro,Justicia,40.41995,-3.69764,Entire home/apt,115,64,Centro,Justicia
4,26825,114340,Agustina,Arganzuela,Legazpi,40.38985,-3.69011,Private room,25,136,Arganzuela,Legazpi


- Old columns (district and neighbourhood) are removed.

In [19]:
stay = stay.drop(['disctrict', 'neighbourhood'], axis = 1 )
stay.head()

,id,host_id,host_name,latitude,longitude,room_type,price,number_of_reviews,disctricts,neighbourhoods
0,6369,13660,Simon,40.45628,-3.67763,Private room,70,65,Chamartín,Hispanoamérica
1,21853,83531,Abdel,40.40341,-3.74084,Private room,17,33,Latina,Cármenes
2,24805,101471,Iraido,40.42202,-3.70395,Entire home/apt,80,2,Centro,Universidad
3,24836,101653,Tenty,40.41995,-3.69764,Entire home/apt,115,64,Centro,Justicia
4,26825,114340,Agustina,40.38985,-3.69011,Private room,25,136,Arganzuela,Legazpi


- We reorganise the columns.

In [22]:
stay = stay[['id','host_id', 'host_name', 'room_type', 'disctricts', 'neighbourhoods', 'price', 'number_of_reviews', 'latitude', 'longitude']]
stay.head()

,id,host_id,host_name,room_type,disctricts,neighbourhoods,price,number_of_reviews,latitude,longitude
0,6369,13660,Simon,Private room,Chamartín,Hispanoamérica,70,65,40.45628,-3.67763
1,21853,83531,Abdel,Private room,Latina,Cármenes,17,33,40.40341,-3.74084
2,24805,101471,Iraido,Entire home/apt,Centro,Universidad,80,2,40.42202,-3.70395
3,24836,101653,Tenty,Entire home/apt,Centro,Justicia,115,64,40.41995,-3.69764
4,26825,114340,Agustina,Private room,Arganzuela,Legazpi,25,136,40.38985,-3.69011


- The new shape of the dataset.

In [23]:
stay.shape

(20837, 10)

In [24]:
stay

,id,host_id,host_name,room_type,disctricts,neighbourhoods,price,number_of_reviews,latitude,longitude
0,6369,13660,Simon,Private room,Chamartín,Hispanoamérica,70,65,40.45628,-3.67763
1,21853,83531,Abdel,Private room,Latina,Cármenes,17,33,40.40341,-3.74084
2,24805,101471,Iraido,Entire home/apt,Centro,Universidad,80,2,40.42202,-3.70395
3,24836,101653,Tenty,Entire home/apt,Centro,Justicia,115,64,40.41995,-3.69764
4,26825,114340,Agustina,Private room,Arganzuela,Legazpi,25,136,40.38985,-3.69011
...,...,...,...,...,...,...,...,...,...,...
20832,38725567,226958922,CristÃ³bal,Entire home/apt,Chamartín,Castilla,100,0,40.47847,-3.68395
20833,38726351,178608863,Rostyslav,Private room,Puente de Vallecas,Palomeras Sureste,24,0,40.38533,-3.63968
20834,38727654,182062359,Jose Maciel,Entire home/apt,Centro,Universidad,75,0,40.42182,-3.70336
20835,38728727,31425029,Valeska,Entire home/apt,Centro,Embajadores,45,0,40.40896,-3.70107


In [25]:
stay.to_csv("madrid_stay.csv", encoding='utf-8-sig')